# take a look at A Tool

In [1]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
{'name': tool.name,
 'description': tool.description,
 'input': tool.args,
 'return_direct': tool.return_direct,
 }

{'name': 'wikipedia',
 'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.',
 'input': {'query': {'title': 'Query', 'type': 'string'}},
 'return_direct': False}

In [2]:
tool.run({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

# function call

In [2]:
from openai import OpenAI
from datetime import datetime
import json
import os

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"), # 如果您没有配置环境变量，请在此处用您的API Key进行替换
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope SDK的base_url
)

## define tools list

In [3]:
# 定义工具列表，模型在选择使用哪个工具时会参考工具的name和description
tools = [
    # 工具1 获取当前时刻的时间
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "当你想知道现在的时间时非常有用。",
            "parameters": {}  # 因为获取当前时间无需输入参数，因此parameters为空字典
        }
    },  
    # 工具2 获取指定城市的天气
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "当你想查询指定城市的天气时非常有用。",
            "parameters": {  # 查询天气时需要提供位置，因此参数设置为location
                        "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市或县区，比如北京市、杭州市、余杭区等。"
                    }
                }
            },
            "required": [
                "location"
            ]
        }
    }
]

# 模拟天气查询工具。返回结果示例：“北京今天是晴天。”
def get_current_weather(location):
    return f"{location}今天是雨天。 "

# 查询当前时间的工具。返回结果示例：“当前时间：2024-04-15 17:15:18。“
def get_current_time():
    # 获取当前日期和时间
    current_datetime = datetime.now()
    # 格式化当前日期和时间
    formatted_time = current_datetime.strftime('%Y-%m-%d %H:%M:%S')
    # 返回格式化后的当前时间
    return f"当前时间：{formatted_time}。"

## Invoke LLM with function calling

In [9]:
# 封装模型响应函数
def get_response(messages):
    completion = client.chat.completions.create(
        model="qwen-plus",
        messages=messages,
        tools=tools
        )
    return completion.model_dump()

def call_with_messages():
    print('\n')
    messages = [
            {
                "content": "杭州和北京天气怎么样？现在几点了？",
                "role": "user"
            }
    ]
    print("-"*60)
    # 模型的第一轮调用
    i = 1
    first_response = get_response(messages)
    assistant_output = first_response['choices'][0]['message']
    print(f"\n第{i}轮大模型输出信息：{first_response}\n")
    if  assistant_output['content'] is None:
        assistant_output['content'] = ""
    messages.append(assistant_output)
    # 如果不需要调用工具，则直接返回最终答案
    if assistant_output['tool_calls'] == None:  # 如果模型判断无需调用工具，则将assistant的回复直接打印出来，无需进行模型的第二轮调用
        print(f"无需调用工具，我可以直接回复：{assistant_output['content']}")
        return
    # 如果需要调用工具，则进行模型的多轮调用，直到模型判断无需调用工具
    while assistant_output['tool_calls'] != None:
        # 如果判断需要调用查询天气工具，则运行查询天气工具
        if assistant_output['tool_calls'][0]['function']['name'] == 'get_current_weather':
            tool_info = {"name": "get_current_weather", "role":"tool"}
            # 提取位置参数信息
            arguments = assistant_output['tool_calls'][0]['function']['arguments']
            if 'properties' in arguments:
                location = arguments['properties']['location']
            else:
                location = arguments['location']
            tool_info['content'] = get_current_weather(location)
        # 如果判断需要调用查询时间工具，则运行查询时间工具
        elif assistant_output['tool_calls'][0]['function']['name'] == 'get_current_time':
            tool_info = {"name": "get_current_time", "role":"tool"}
            tool_info['content'] = get_current_time()
        print(f"工具输出信息：{tool_info['content']}\n")
        print("-"*60)
        messages.append(tool_info)
        assistant_output = get_response(messages)['choices'][0]['message']
        if  assistant_output['content'] is None:
            assistant_output['content'] = ""
        messages.append(assistant_output)
        i += 1
        print(f"第{i}轮大模型输出信息：{assistant_output}\n")
    print(f"最终答案：{assistant_output['content']}")


call_with_messages()



------------------------------------------------------------

第1轮大模型输出信息：{'id': 'chatcmpl-aaf6e713-622a-9a7c-9024-b3d5056830f9', 'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': '', 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_88713c110c0b4442af9039', 'function': {'arguments': '{"properties": {"location": "杭州市"}}', 'name': 'get_current_weather'}, 'type': 'function', 'index': 0}]}}], 'created': 1721742531, 'model': 'qwen-plus', 'object': 'chat.completion', 'system_fingerprint': None, 'usage': {'completion_tokens': 21, 'prompt_tokens': 230, 'total_tokens': 251}}

工具输出信息：杭州市今天是雨天。 

------------------------------------------------------------
第2轮大模型输出信息：{'content': '', 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_e302c8e9f6f64cdebe2720', 'function': {'arguments': '{"properties": {"location": "北京市"}}', 'name': 'get_current_weather'}, 'type': 'function', 'index': 0}]}

工具输出信息：北京市今天是雨天。 



# LangChain Tools as OpenAI Functions
## Defining Custom Tools
### @tool decorator

In [1]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def get_current_weather(location: str) -> str:
    """当你想查询指定城市的天气时非常有用。"""
    return f"{location}今天是雨天。"

In [20]:
get_current_weather.name, get_current_weather.description, get_current_weather.args

('get_current_weather',
 'get_current_weather(location: str) -> str - 当你想查询指定城市的天气时非常有用。',
 {'location': {'title': 'Location', 'type': 'string'}})

### tool decorator with JSON args

In [21]:
class InputSchema(BaseModel):
    location: str = Field(description="城市或县区，比如北京市、杭州市、余杭区等。")

@tool("get_current_weather", args_schema=InputSchema)
def get_current_weather(location: str):
    """当你想查询指定城市的天气时非常有用。"""
    return f"{location}今天是雨天。"

In [22]:
get_current_weather.name, get_current_weather.description, get_current_weather.args

('get_current_weather',
 'get_current_weather(location: str) - 当你想查询指定城市的天气时非常有用。',
 {'location': {'title': 'Location',
   'description': '城市或县区，比如北京市、杭州市、余杭区等。',
   'type': 'string'}})

### Subclass BaseTool

In [24]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class InputSchema(BaseModel):
    location: str = Field(description="城市或县区，比如北京市、杭州市、余杭区等。")

class GetCurrentWeatherTool(BaseTool):
    name = "get_current_weather"
    description = "当你想查询指定城市的天气时非常有用。"
    args_schema: Type[BaseModel] = InputSchema

    def _run(
        self, location: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return f"{location}今天是雨天。"

In [25]:
get_current_weather = GetCurrentWeatherTool()
get_current_weather.name, get_current_weather.description, get_current_weather.args

('get_current_weather',
 '当你想查询指定城市的天气时非常有用。',
 {'location': {'title': 'Location',
   'description': '城市或县区，比如北京市、杭州市、余杭区等。',
   'type': 'string'}})

### StructuredTool dataclass

In [26]:
class InputSchema(BaseModel):
    location: str = Field(description="城市或县区，比如北京市、杭州市、余杭区等。")

def get_current_weather_func(location: str):
    """当你想查询指定城市的天气时非常有用。"""
    return f"{location}今天是雨天。"

get_current_weather = StructuredTool.from_function(
    func=get_current_weather_func,
    name="get_current_weather",
    description="当你想查询指定城市的天气时非常有用。",
    args_schema=InputSchema
)

In [27]:
get_current_weather.name, get_current_weather.description, get_current_weather.args

('get_current_weather',
 'get_current_weather(location: str) - 当你想查询指定城市的天气时非常有用。',
 {'location': {'title': 'Location',
   'description': '城市或县区，比如北京市、杭州市、余杭区等。',
   'type': 'string'}})

## Tools as OpenAI Functions 

In [2]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from tools.function_calling import convert_to_openai_tool

In [3]:
class WeatherSchema(BaseModel):
    location: str = Field(description="城市或县区，比如北京市、杭州市、余杭区等。")

@tool("get_current_weather", args_schema=WeatherSchema)
def get_current_weather(location: str):
    """当你想查询指定城市的天气时非常有用。"""
    return f"{location}今天是雨天。"


# 查询当前时间的工具。返回结果示例：“当前时间：2024-04-15 17:15:18。“
@tool("get_current_time")
def get_current_time():
    """当你想知道现在的时间时非常有用。"""
    # 获取当前日期和时间
    current_datetime = datetime.now()
    # 格式化当前日期和时间
    formatted_time = current_datetime.strftime('%Y-%m-%d %H:%M:%S')
    # 返回格式化后的当前时间
    return f"当前时间：{formatted_time}。"

functions = [get_current_weather, get_current_time]
tools = [convert_to_openai_tool(t) for t in functions]

In [4]:
tools

[{'type': 'function',
  'function': {'name': 'get_current_weather',
   'description': 'get_current_weather(location: str) - 当你想查询指定城市的天气时非常有用。',
   'parameters': {'type': 'object',
    'properties': {'location': {'description': '城市或县区，比如北京市、杭州市、余杭区等。',
      'type': 'string'}},
    'required': ['location']}}},
 {'type': 'function',
  'function': {'name': 'get_current_time',
   'description': 'get_current_time() - 当你想知道现在的时间时非常有用。',
   'parameters': {'type': 'object', 'properties': {}}}}]

In [13]:
from tongyi.chat_model import CustomChatTongyi

model = CustomChatTongyi(model='qwen-plus')

messages = [HumanMessage(content="杭州和北京天气怎么样？现在几点了？")]

assistant_output = model.invoke(messages, tools=tools)
assistant_output

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': '{"location": "杭州市"}'}, 'index': 0, 'id': 'call_c72afbea224b4de6bc3957', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '6ee616af-aa17-9600-bd04-8bcaf00e2f56', 'token_usage': {'input_tokens': 261, 'output_tokens': 18, 'total_tokens': 279}}, id='run-63bc17f2-8f36-41c2-b890-96d3a7ab7376-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': '杭州市'}, 'id': 'call_c72afbea224b4de6bc3957'}])

In [14]:
functions_to_call = {'get_current_weather': get_current_weather, 'get_current_time': get_current_time}

while assistant_output.tool_calls:
    messages.append(assistant_output)
    for tool in assistant_output.tool_calls:
        args = tool['args'] if 'properties' not in tool['args'] else tool['args']['properties']
        tool_content = functions_to_call[tool['name']].invoke(args)
        messages.append(ToolMessage(name=tool['name'], 
                                    tool_call_id=tool['id'], 
                                    content=tool_content)
                        )
        print(f"工具输出信息：{tool_content}\n")
        print("-"*60)
    
    assistant_output = model.invoke(messages, tools=tools)
    print(f"大模型输出信息：{assistant_output}\n")

工具输出信息：杭州市今天是雨天。

------------------------------------------------------------
大模型输出信息：content='' additional_kwargs={'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': '{"location": "北京市"}'}, 'index': 0, 'id': 'call_6dfa56e25ab84322b5cc2f', 'type': 'function'}]} response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '88873f57-7d98-962f-af86-7e4b3e2c9a80', 'token_usage': {'input_tokens': 294, 'output_tokens': 20, 'total_tokens': 314}} id='run-6d079aaa-9469-4be9-9c22-7a0f97e9dd85-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': '北京市'}, 'id': 'call_6dfa56e25ab84322b5cc2f'}]

工具输出信息：北京市今天是雨天。

------------------------------------------------------------
大模型输出信息：content='' additional_kwargs={'tool_calls': [{'function': {'name': 'get_current_time', 'arguments': '{"properties": {}}'}, 'index': 0, 'id': 'call_97f6441d9b9d4503ae2dea', 'type': 'function'}]} response_metadata={'model_name': 'qwen-plus', 'finish_reason': 

### bind_tools

In [7]:
model_with_tools = model.bind_tools(functions)

messages = [HumanMessage(content="杭州和北京天气怎么样？现在几点了？")]

assistant_output = model_with_tools.invoke(messages)
assistant_output

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': '{"location": "杭州市"}'}, 'index': 0, 'id': 'call_72d4e08d6ddc4fe5974f72', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '86640a08-c7fd-9c0a-a350-0deb481bf5d1', 'token_usage': {'input_tokens': 261, 'output_tokens': 18, 'total_tokens': 279}}, id='run-c663249c-2967-41e6-a695-88269dcc4446-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': '杭州市'}, 'id': 'call_72d4e08d6ddc4fe5974f72'}])

# Agents
## Quickstart

In [4]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name='chat_history', optional=True),
     HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
     MessagesPlaceholder(variable_name='agent_scratchpad')]
    )

In [5]:
from datetime import datetime
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool

class WeatherSchema(BaseModel):
    location: str = Field(description="城市或县区，比如北京市、杭州市、余杭区等。")

@tool("get_current_weather", args_schema=WeatherSchema)
def get_current_weather(location: str):
    """当你想查询指定城市的天气时非常有用。"""
    return f"{location}今天是雨天。"


# 查询当前时间的工具。返回结果示例：“当前时间：2024-04-15 17:15:18。“
@tool("get_current_time")
def get_current_time():
    """当你想知道现在的时间时非常有用。"""
    # 获取当前日期和时间
    current_datetime = datetime.now()
    # 格式化当前日期和时间
    formatted_time = current_datetime.strftime('%Y-%m-%d %H:%M:%S')
    # 返回格式化后的当前时间
    return f"当前时间：{formatted_time}。"

tools = [get_current_weather, get_current_time]

In [3]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

from tongyi.chat_model import CustomChatTongyi

model = CustomChatTongyi(model='qwen-plus')

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({'input': '杭州和北京天气怎么样？现在几点了？'})



> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'location': '杭州市'}`


杭州市今天是雨天。
Invoking: `get_current_weather` with `{'location': '北京市'}`


北京市今天是雨天。
Invoking: `get_current_time` with `{}`


当前时间：2024-07-30 15:35:17。目前杭州市和北京市的天气都是雨天。现在的时间是2024年7月30日15点35分17秒。记得带伞出门哦！

> Finished chain.


{'input': '杭州和北京天气怎么样？现在几点了？',
 'output': '目前杭州市和北京市的天气都是雨天。现在的时间是2024年7月30日15点35分17秒。记得带伞出门哦！'}

## XML Agent

In [1]:
from langchain import hub
prompt = hub.pull("hwchase17/xml-agent-convo")
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], template="You are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:\n\n<tool>search</tool><tool_input>weather in SF</tool_input>\n<observation>64 degrees</observation>\n\nWhen you are done, respond with a final answer between <final_answer></final_answer>. For example:\n\n<final_answer>The weather in SF is 64 degrees</final_answer>\n\nBegin!\n\nPrevious Conversation:\n{chat_history}\n\nQuestion: {input}\n{agent_scratchpad}"))]

In [3]:
from datetime import datetime
from langchain.pydantic_v1 import BaseModel, Field

from tools.tools import ModifyStructuredTool

class WeatherSchema(BaseModel):
    location: str = Field(description="城市或县区，比如北京市、杭州市、余杭区等。")

def get_current_weather_func(location: str):
    """当你想查询指定城市的天气时非常有用。"""
    return f"{location}今天是雨天。"


get_current_weather = ModifyStructuredTool.from_function(
    func=get_current_weather_func,
    name="get_current_weather",
    description="当你想查询指定城市的天气时非常有用。",
    args_schema=WeatherSchema
)


def get_current_time_func():
    """当你想知道现在的时间时非常有用。"""
    # 获取当前日期和时间
    current_datetime = datetime.now()
    # 格式化当前日期和时间
    formatted_time = current_datetime.strftime('%Y-%m-%d %H:%M:%S')
    # 返回格式化后的当前时间
    return f"当前时间：{formatted_time}。"


get_current_time = ModifyStructuredTool.from_function(
    func=get_current_time_func,
    name="get_current_time",
    description="当你想知道现在的时间时非常有用。",
)

tools = [get_current_weather, get_current_time]

In [4]:
from langchain.agents import create_xml_agent
from langchain.agents import AgentExecutor

from tongyi.chat_model import CustomChatTongyi

model = CustomChatTongyi(model='qwen-plus')

agent = create_xml_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [5]:
agent_executor.invoke({'input': '杭州和北京天气怎么样？现在几点了？'})



> Entering new AgentExecutor chain...
<tool>get_current_weather</tool><tool_input>杭州杭州今天是雨天。<tool>get_current_weather</tool><tool_input>北京北京今天是雨天。<tool>get_current_time</tool><tool_input>当前时间：2024-07-31 16:59:52。<final_answer>杭州和北京今天的天气都是雨天。现在的时间是2024年7月31日16点59分52秒。</final_answer>

> Finished chain.


{'input': '杭州和北京天气怎么样？现在几点了？',
 'output': '杭州和北京今天的天气都是雨天。现在的时间是2024年7月31日16点59分52秒。',
 'intermediate_steps': [(AgentAction(tool='get_current_weather', tool_input='杭州', log='<tool>get_current_weather</tool><tool_input>杭州'),
   '杭州今天是雨天。'),
  (AgentAction(tool='get_current_weather', tool_input='北京', log='<tool>get_current_weather</tool><tool_input>北京'),
   '北京今天是雨天。'),
  (AgentAction(tool='get_current_time', tool_input='', log='<tool>get_current_time</tool><tool_input>'),
   '当前时间：2024-07-31 16:59:52。')]}

## Agent Iterator

In [7]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

for i, step in enumerate(agent_executor.iter({'input': '杭州和北京天气怎么样？现在几点了？'})):
    print(f'Agent的第 {i+1} 步输出:')
    if output := step.get("intermediate_step"):
        action, value = output[0]
        print(f'\taction: {action}')
        print(f'\tvalue: {value}')
    else:
        print(step['output'])

Agent的第 1 步输出:
	action: tool='get_current_weather' tool_input={'location': '杭州市'} log="\nInvoking: `get_current_weather` with `{'location': '杭州市'}`\n\n\n" message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_c41f51d0064b46e1af76e3call_c41f51d0064b46e1af76e3call_c41f51d0064b46e1af76e3call_c41f51d0064b46e1af76e3', 'type': 'functionfunctionfunctionfunction', 'function': {'name': 'get_current_weather', 'arguments': '{"location": "杭州市"}'}}]}, response_metadata={'finish_reason': 'tool_calls', 'request_id': '43be987d-0fc2-9a71-8f34-178dbacc5db6', 'token_usage': {'input_tokens': 261, 'output_tokens': 18, 'total_tokens': 279}, 'model_name': 'qwen-plus'}, id='run-dd8f971c-95ae-47fb-b820-3cb3d3ed158f', tool_calls=[{'name': 'get_current_weather', 'args': {'location': '杭州市'}, 'id': 'call_c41f51d0064b46e1af76e3call_c41f51d0064b46e1af76e3call_c41f51d0064b46e1af76e3call_c41f51d0064b46e1af76e3'}], tool_call_chunks=[{'name': 'get_current_weather', 'args': '{

## Structured Output

In [6]:
from typing import List, Dict

from langchain_core.agents import AgentActionMessageLog, AgentFinish, AgentAction
from langchain.agents.output_parsers.tools import parse_ai_message_to_tool_action
from langchain_core.pydantic_v1 import BaseModel, Field


class Response(BaseModel):
    """最终的回答如果包含天气、时间等信息，则需要调用该工具"""

    weather: Dict[str, str] = Field(
        description='最终的回答中不同城市对应的天气，比如北京的天气为阴天，则返回`{"北京": "阴天"}`。如果不存在天气信息则忽略', default={})
    current_time: str = Field(
        description="最终的回答中的当前时间，如果不存在则忽略", default="")


def parse(output):
    name = None
    # 判断调用的工具是否为 Response
    if output.tool_calls:
        tool_calls = output.tool_calls[0]
        name = tool_calls["name"]
        inputs = tool_calls["args"]
    elif output.additional_kwargs.get("tool_calls"):
        tool_calls = output.additional_kwargs["tool_calls"][0]
        name = tool_calls["name"]
        inputs = tool_calls["args"]
    
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(tool_calls))

    return parse_ai_message_to_tool_action(output)

In [7]:
# prompt & tools的定义在上面[Agents-Quickstart]小节,不再重复编写
from langchain.agents.format_scratchpad import format_to_tool_messages
from langchain.agents import AgentExecutor
from tongyi.chat_model import CustomChatTongyi

model = CustomChatTongyi(model='qwen-plus')
llm_with_tools = model.bind_tools(tools + [Response])

agent = (
    {
        "input": lambda x: x["input"],
        # Format agent scratchpad from intermediate steps
        "agent_scratchpad": lambda x: format_to_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | parse
)

In [10]:
agent_executor = AgentExecutor(tools=tools, agent=agent, verbose=True)

agent_executor.invoke(
    {"input": "杭州和北京天气怎么样？现在几点了？"},
    return_only_outputs=True,
)



> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'location': '杭州市'}`


杭州市今天是雨天。
Invoking: `get_current_weather` with `{'location': '北京市'}`


北京市今天是雨天。
Invoking: `get_current_time` with `{}`


当前时间：2024-08-01 22:19:30。{'name': 'Response', 'args': {'current_time': '2024-08-01 22:19:30', 'weather': {'杭州市': '雨天', '北京市': '雨天'}}, 'id': 'call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785call_8201119de6cb4077b0a785'}

> Finished chain.


{'current_time': '2024-08-01 22:19:30', 'weather': {'杭州市': '雨天', '北京市': '雨天'}}